In [2]:
import requests

url = "https://landy-ai.vercel.app/invoke"

payload = {
    "user_input": "car showroom ",
    'thread_id': 'a'
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print(response.status_code)
print(response.json())
# Expected Output: {'state_id': '05f25ca0-0647-44eb-9d4e-de466afef435', 'user_input': 'car showroom ', 'preferences': {'buy_rent': '', 'location': 'Klang Valley', 'size': '', 'business_nature': 'Car showroom', 'power_requirement': '', 'floor_loading_capacity': '', 'ceiling_height': '', 'budget': ''}, 'recommended_listing': [{'location': 'Chan Sow Lin, near TRX, KL', 'built_up_area_sqft': '35000', 'land_area_sqft': '40000', 'zoning_type': 'warehouse zone', 'power_capacity_kva': '', 'floor_loading_ton_per_sqm': '', 'clear_height_m': '', 'asking_price': '', 'source': 'https://www.industrialprop.com.my/property/kl-centre-car-showroom-warehouse-factory-for-rent/'}, {'location': 'Chan Sow Lin, Kuala Lumpur', 'built_up_area_sqft': '10673', 'land_area_sqft': '12572', 'zoning_type': '', 'power_capacity_kva': '', 'floor_loading_ton_per_sqm': '', 'clear_height_m': '', 'asking_price': '9000000', 'source': 'https://www.industrialprop.com.my/?post_type=property&#038;p=1823'}, {'location': 'Chan Sow Lin, Kuala Lumpur', 'built_up_area_sqft': '20000', 'land_area_sqft': '20000', 'zoning_type': 'Commercial and Industrial', 'power_capacity_kva': '', 'floor_loading_ton_per_sqm': '', 'clear_height_m': '', 'asking_price': '55000', 'source': 'https://www.industrialprop.com.my/property/chan-sow-lin-kuala-lumpur-commercial-lot-for-rent/'}, {'location': 'ETP 3, near Port Klang', 'built_up_area_sqft': '14616', 'land_area_sqft': '20169', 'zoning_type': '', 'power_capacity_kva': '288', 'floor_loading_ton_per_sqm': '3', 'clear_height_m': '14.63', 'asking_price': '', 'source': 'https://www.industrialprop.com.my/?post_type=property&#038;p=1288'}, {'location': 'Bandar Sunway, Subang Jaya', 'built_up_area_sqft': '14000', 'land_area_sqft': '17000', 'zoning_type': '', 'power_capacity_kva': '', 'floor_loading_ton_per_sqm': '', 'clear_height_m': '', 'asking_price': '', 'source': 'https://www.industrialprop.asia/?post_type=property&#038;p=536'}], 'graph_output': 'Here are suitable listings in Klang Valley ideal for a car showroom. See details for options in prime locations.'}



200
{'state_id': 'b7fdea57-198f-49b5-bf26-dc6a0513f2e3', 'user_input': 'car showroom ', 'preferences': {'buy_rent': '', 'location': 'Klang Valley', 'size': '', 'business_nature': 'Car showroom', 'power_requirement': '', 'floor_loading_capacity': '', 'ceiling_height': '', 'budget': ''}, 'recommended_listing': [{'location': 'Chan Sow Lin, near TRX, KL', 'built_up_area_sqft': '35000', 'land_area_sqft': '40000', 'zoning_type': 'warehouse zone', 'power_capacity_kva': '', 'floor_loading_ton_per_sqm': '', 'clear_height_m': '', 'asking_price': '', 'source': 'https://www.industrialprop.com.my/property/kl-centre-car-showroom-warehouse-factory-for-rent/'}, {'location': 'Chan Sow Lin, Kuala Lumpur', 'built_up_area_sqft': '10673', 'land_area_sqft': '12572', 'zoning_type': '', 'power_capacity_kva': '', 'floor_loading_ton_per_sqm': '', 'clear_height_m': '', 'asking_price': '9000000', 'source': 'https://www.industrialprop.com.my/?post_type=property&#038;p=1823'}, {'location': 'Taman Dutamas Balakong (

In [ ]:
from utility.property_listing_init import get_property_listing


ModuleNotFoundError: No module named 'pymongo'

In [ ]:
import pymongo

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
from typing_extensions import TypedDict, Literal, Dict, Optional, List
from langchain_core.prompts import ChatPromptTemplate

class OverallState(TypedDict):
    user_input: str
    preferences: Optional[Dict[str, str]]
    recommended_listing: List[str]
    graph_output: str


In [ ]:
from prompt import get_system_prompt
from langgraph.types import interrupt, Command
from langgraph.graph import StateGraph, START, END

class Preference(TypedDict):
    buy_rent: Optional[Literal['Buy', 'Rent']]
    location: Optional[str]
    size: Optional[str]
    business_nature: Optional[str]
    power_requirement: Optional[str]
    floor_loading_capacity: Optional[str]
    ceiling_height: Optional[str]
    budget: Optional[str]

class Planner(TypedDict):
    planner_decision: Literal['final_output', 'property_listing_lookup']
    rephrased_query: str
    preferences: Preference
    messages: str

def Landy_Planner(OverallState):  
    user_input = OverallState['user_input']  

    system_prompt = get_system_prompt()

    template = ChatPromptTemplate(
        [
            ("system", system_prompt),
            ("human", "{user_input}"),
        ]
    )

    chain = template | llm.with_structured_output(Planner)

    response = chain.invoke({
        "user_input": user_input
    })

    if response['planner_decision'] == 'final_output':
        goto = END
    else:
        goto = "Property_Lookup_Agent"
    
    return Command(
        update = {
        'graph_output': response['messages'],
        'preferences': response['preferences']
        },
        goto = goto
        )

In [ ]:
from prompt import get_property_search_prompt
import json

class Listing(TypedDict):
    location: Optional[str]
    built_up_area_sqft: Optional[str]
    land_area_sqft: Optional[str]
    zoning_type: Optional[str]
    power_capacity_kva: Optional[str]
    floor_loading_ton_per_sqm: Optional[str]
    clear_height_m: Optional[str]
    asking_price: Optional[str]
    source: Optional[str]
    
class FinalOutput(TypedDict):
    recommended_listing = Optional[List[Listing]]
    messages: str

# Open the file in read mode ('r')
with open('listing.json', 'r') as file:
    # Use json.load() to convert the file content to a Python dictionary
    listing = json.load(file)

def Property_Lookup_Agent(state):    
    property_search_prompt = get_property_search_prompt()

    template = ChatPromptTemplate(
        [
            ("system", property_search_prompt),
            ("human", "{user_preferences}"),
        ]
    )

    chain = template | llm.with_structured_output(FinalOutput)
    response = chain.invoke({
        "listing": listing,
        "user_preferences": OverallState['preferences'],
    })

    print(response.get('recommended_listing', ''))
    
    return Command(
        update = {
            'graph_output': response['messages'],
            'recommended_listing': response['recommended_listing']
            },
        goto = END)

In [ ]:

builder = StateGraph(OverallState)
builder.add_node("Landy_Planner", Landy_Planner)
builder.add_node("Property_Lookup_Agent", Property_Lookup_Agent)

builder.add_edge(START, "Landy_Planner")
builder.add_edge("Property_Lookup_Agent", END)

graph = builder.compile()


In [ ]:
graph.invoke({"user_input": "Hi"})
